<!-- # CNN autoencoder and Clustering from MTRX data

Use this notebook to load Scienta Omicron Matrix format SPM data and create standardised images for machine learning training and analysis. The code can generate both JPG image data, useful for manually checking the data, and windowed numpy data that can be loaded into ML models. 

The notebook then creates an autoencoder for training on a large dataset, followed by KMEANS clustering. 

**Author**: Steven R. Schofield  
**Created**: November, 2024 -->

# CASTEP - Si 001 surface
## Steven R. Schofield (Universtiy College London) May 2025

### Load required modules

In [1]:
import sys
from pathlib import Path

# Define candidate paths using Path objects
module_path_list = [
    Path('/Users/steven/academic-iCloud/Python/modules'),
    Path('/hpc/srs/Python/modules')
]

data_path_list = [
    Path('/Users/steven/academic-iCloud/Calculations/castep/'),
    Path('/hpc/srs/castep')
]

# Resolve actual paths
module_path = next((p for p in module_path_list if p.exists()), None)
data_path = next((p for p in data_path_list if p.exists()), None)

# Check and report missing paths
if module_path is None:
    print("Error: Could not locate a valid module path.")
if data_path is None:
    print("Error: Could not locate a valid data path.")

if module_path is None or data_path is None:
    sys.exit(1)

# Add module_path to sys.path if needed
if str(module_path) not in sys.path:
    sys.path.insert(0, str(module_path))

# Print resolved paths
print(f"module_path = {module_path}")
print(f"data_path = {data_path}")

module_path = /Users/steven/academic-iCloud/Python/modules
data_path = /Users/steven/academic-iCloud/Calculations/castep


In [2]:
# # Ensure modules are reloaded 
%load_ext autoreload
%autoreload 2

# Import standard modules
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt

# Import custom module
import SRSCALCUTILS.castep_tools as ct

from IPython.display import display

### Job paths

In [3]:
# Parameters for windows creation
# General
job_folder = 'work-2025/si001_surf'
job_path = data_path / job_folder
verbose = False             # Set this True to print out more information

# Print resolved paths
print(f"job_path = {job_path}")

job_path = /Users/steven/academic-iCloud/Calculations/castep/work-2025/si001_surf


In [4]:
# If it doesn’t exist, create it (including any missing parent folders)
if not os.path.isdir(job_path):
    os.makedirs(job_path, exist_ok=True)

# Now safely switch into it
os.chdir(job_path)

print("Now in:", os.getcwd())
print("Files:", os.listdir())

Now in: /Users/steven/Library/Mobile Documents/com~apple~CloudDocs/academic/Calculations/castep/work-2025/si001_surf
Files: ['si001_test.cell', 'si001_test.param']


### Optimised params

In [5]:
# Set up the param and cell files for the calculation

filename = "si001_surf"
title = "silicon 001 surface tests"

params = {'task' : 'geomopt',
          'xc_functional'           : 'PBE',
          'cut_off_energy'          : 700,
          'spin_polarised'          : 'false',
          'write_cell_structure'    : 'true',
          'charge'                  : 0,
          'nextra_bands'            : 4,
          'geom_energy_tol'         : 0.00005,          # default: 0.00005 eV 
          'geom_disp_tol'           : 0.002,            # default: 0.002 Ang.
          'geom_max_iter'           : 100,              # default: 30
          'geom_convergence_win'    : 2,                # default: 2
          'max_scf_cycles'          : 300               # default: 30
          }              

lattice_cart = np.array([
            [3.8641976,     0.0,     0.0],
            [0.0,     3.8641976,     0.0],
            [0.0,     0.0,     5.4648012]
        ])

positions_frac = np.array([
            ['H', 0.0,  0.0,  0.0],
            ['Si', 0.5,  0.0,  0.25],
            ['Si', 0.5,  0.5,  0.5],
            ['Si', 0.0,  0.5,  0.75],
        ])

cell_constraints = np.array([
            [1,  1,  2],
            [0,  0,  0]
        ])

kpoints_mp_grid = np.array([8, 8, 4])

fix_all_ions = True
symmetry_generate=False
symmetry_tol = 0.01

In [6]:
na=2
nb=2
nc=1

positions_frac = ct.create_supercell_from_fractional_coords(
    positions_frac,
    na=na,
    nb=nb,
    nc=nc)

print(positions_frac)

[['Si' 0.5 0.75 0.75]
 ['Si' 0.0 0.75 0.75]
 ['Si' 0.5 0.25 0.75]
 ['Si' 0.0 0.25 0.75]
 ['Si' 0.75 0.75 0.5]
 ['Si' 0.25 0.75 0.5]
 ['Si' 0.75 0.25 0.5]
 ['Si' 0.25 0.25 0.5]
 ['Si' 0.75 0.5 0.25]
 ['Si' 0.25 0.5 0.25]
 ['Si' 0.75 0.0 0.25]
 ['Si' 0.25 0.0 0.25]
 ['H' 0.5 0.5 0.0]
 ['H' 0.0 0.5 0.0]
 ['H' 0.5 0.0 0.0]
 ['H' 0.0 0.0 0.0]]


In [61]:
import numpy as np



# Select atoms with z < 3 Angstrom or atom Si
cond = "z < 2"
exclude = None
include = None
result = ct.select_atoms_by_region(positions_frac, lattice_cart, cond,exclude=exclude, include=include)
print(result)
cons = ct.write_ionic_constraints(result)
for line in cons:
    print(line)

[[1 False 'Si' 0.5 0.75 0.75]
 [2 False 'Si' 0.0 0.75 0.75]
 [3 False 'Si' 0.5 0.25 0.75]
 [4 False 'Si' 0.0 0.25 0.75]
 [5 False 'Si' 0.75 0.75 0.5]
 [6 False 'Si' 0.25 0.75 0.5]
 [7 False 'Si' 0.75 0.25 0.5]
 [8 False 'Si' 0.25 0.25 0.5]
 [9 True 'Si' 0.75 0.5 0.25]
 [10 True 'Si' 0.25 0.5 0.25]
 [11 True 'Si' 0.75 0.0 0.25]
 [12 True 'Si' 0.25 0.0 0.25]
 [13 True 'H' 0.5 0.5 0.0]
 [14 True 'H' 0.0 0.5 0.0]
 [15 True 'H' 0.5 0.0 0.0]
 [16 True 'H' 0.0 0.0 0.0]]
%BLOCK ionic_constraints
    1 Si         1    1.00000000    0.00000000    0.00000000
    2 Si         1    0.00000000    1.00000000    0.00000000
    3 Si         1    0.00000000    0.00000000    1.00000000
    4 Si         2    1.00000000    0.00000000    0.00000000
    5 Si         2    0.00000000    1.00000000    0.00000000
    6 Si         2    0.00000000    0.00000000    1.00000000
    7 Si         3    1.00000000    0.00000000    0.00000000
    8 Si         3    0.00000000    1.00000000    0.00000000
    9 Si         3 

%BLOCK ionic_constraints
    1 Si         1    1.00000000    0.00000000    0.00000000
    2 Si         1    0.00000000    1.00000000    0.00000000
    3 Si         1    0.00000000    0.00000000    1.00000000
    4 Si         2    1.00000000    0.00000000    0.00000000
    5 Si         2    0.00000000    1.00000000    0.00000000
    6 Si         2    0.00000000    0.00000000    1.00000000
    7 Si         3    1.00000000    0.00000000    0.00000000
    8 Si         3    0.00000000    1.00000000    0.00000000
    9 Si         3    0.00000000    0.00000000    1.00000000
   10 Si         4    1.00000000    0.00000000    0.00000000
   11 Si         4    0.00000000    1.00000000    0.00000000
   12 Si         4    0.00000000    0.00000000    1.00000000
   13 H         5    1.00000000    0.00000000    0.00000000
   14 H         5    0.00000000    1.00000000    0.00000000
   15 H         5    0.00000000    0.00000000    1.00000000
   16 H         6    1.00000000    0.00000000    0.00000000
   

In [ ]:
filename = f"si001_test"
castep_path = job_path / f"{filename}.castep"

print('=' * 80 + f'\nCASTEP file: {castep_path}\n' + '=' * 80 + '\n')   

_ = ct.write_param_file(
    params,
    title = title,
    filename=filename,
    path=job_path,
    display_file=False
    )

_ = ct.write_bulk_cell_file(
    title = title,
    path=job_path,
    filename=filename,
    lattice_cart=lattice_cart,
    positions_frac=positions_frac,
    cell_constraints=cell_constraints,
    fix_all_ions=True,
    symmetry_generate=symmetry_generate,
    symmetry_tol = symmetry_tol,
    kpoints_mp_grid=kpoints_mp_grid,
    display_file=False
)



CASTEP file: /Users/steven/academic-iCloud/Calculations/castep/work-2025/si001_surf/si001_test.castep

Wrote param file to: /Users/steven/academic-iCloud/Calculations/castep/work-2025/si001_surf/si001_test.param
Wrote cell file to: /Users/steven/academic-iCloud/Calculations/castep/work-2025/si001_surf/si001_test.cell


In [ ]:
# if str(module_path) == '/hpc/srs/Python/modules':
#     !bash -l -c "mpirun -np 62 castep.mpi {filename}"
# else:
#     !zsh -l -c "castepmpi {filename}"

# unit_cell, a, b, c, alpha, beta, gamma = ct.get_final_lattice_parameters(castep_path)
# energy_optimisation = ct.get_LBFGS_energies(castep_path)
# energies = [val for _, val in energy_optimisation]
# energy = energies[-1]

# print('Optimised energy {} eV'.format(energy))
# print('Optimised lattice constants a,b = {} Ang., c = {} Ang.'.format(a,c))
# print('Unit cell:')
# for line in unit_cell:
#     print(' '*2,np.abs(line))
